In [101]:
import pandas as pd
import numpy as np

## Creating Data

In [102]:
Participant=['Alice','Bob','Cary','Doug','Edna']
Tax_Rate=[3,4,3,2,1]
Fee=[4,3,5,1,1]
Interest_Rate=[3,5,3,3,3]
Quantity_Limit=[2,1,3,3,2]
Price_Limit=[1,1,3,2,3]
d={'Participant':Participant,'Tax_Rate':Tax_Rate,'Fee':Fee,'Interest_Rate':Interest_Rate,'Quantity_Limit':Quantity_Limit,'Price_Limit':Price_Limit}
df=pd.DataFrame(d)
df

,Participant,Tax_Rate,Fee,Interest_Rate,Quantity_Limit,Price_Limit
0,Alice,3,4,3,2,1
1,Bob,4,3,5,1,1
2,Cary,3,5,3,3,3
3,Doug,2,1,3,3,2
4,Edna,1,1,3,2,3


## Transforming to matrix

In [103]:
d=df.iloc[:,1:].values

## Creating Similarity matrix

In [104]:
s = np.zeros(d.shape)
def similarity_m():
    for i in range(d.shape[0]):
        for j in range(d.shape[0]):
                s[i,j]=-sum((d[i]-d[j])**2)
    for i in range(s.shape[0]):
                s[i,i]=s.min()
    return s
similarity_m()

array([[-22.,  -7.,  -6., -12., -17.],
       [ -7., -22., -17., -17., -22.],
       [ -6., -17., -22., -18., -21.],
       [-12., -17., -18., -22.,  -3.],
       [-17., -22., -21.,  -3., -22.]])

## Creating Responsibility matrix

In [105]:
r = np.zeros(d.shape)
a = np.zeros(d.shape)
def responsibility_m():
    for i in range(d.shape[0]):
        for j in range(d.shape[0]):
            r[i,j]=s[i,j]-max(np.delete(a[i,:],j)+np.delete(s[i,:],j))
    return r
responsibility_m()

array([[-16.,  -1.,   1.,  -6., -11.],
       [ 10., -15., -10., -10., -15.],
       [ 11., -11., -16., -12., -15.],
       [ -9., -14., -15., -19.,   9.],
       [-14., -19., -18.,  14., -19.]])

## Creating Availability Matrix

In [106]:
def availability_m():
    for i in range(d.shape[0]):
        for j in range(d.shape[0]):
            m=np.delete(r[:,j],i)
            if i==j:
                a[i,j]=sum(m[m>=0])
            else:
                a[i,j]=min(0,r[j,j]+sum(m[m>=0]))
    return a
availability_m()

array([[ 21., -15., -16.,  -5., -10.],
       [ -5.,   0., -15.,  -5., -10.],
       [ -6., -15.,   1.,  -5., -10.],
       [  0., -15., -15.,  14., -19.],
       [  0., -15., -15., -19.,   9.]])

## Defining the parameters

In [107]:
damp=0.8
n_iter=10

## Creating Criterion Matrix

In [108]:
def criterion_m():
    r = np.zeros(d.shape)
    a = np.zeros(d.shape)
    for i in range(n_iter):
        similarity_m()
        if i==0:
            responsibility_m()
            availability_m()
        else:
            r=(damp*r)+(1-damp)*responsibility_m()
            a=(damp*a)+(1-damp)*availability_m()
        c=r+a
    return c

## Forming Cluster

In [109]:
clusters=[]
for i in criterion_m():
    clusters.append(np.argmax(i))
clusters

[0, 0, 0, 3, 3]